In [2]:
# ------------------------------------------------ #
#                  IMPORT FUNCTIONS                #
# ------------------------------------------------ #
import os
import sys
import datetime

# Get the project root directory (going up one level from Notebooks)
project_root = os.path.dirname(os.getcwd())
# Add the Scripts directory to Python path so we can import modules from individual scripts files
scripts_dir = os.path.join(project_root, 'Scripts')
if scripts_dir not in sys.path:
    sys.path.append(scripts_dir)

# Add functions here from Scripts folder
from download_ao_indices import ReturnAOFetch
from enso_indices import ReturnENSOFetch
from breakup_data import CalculateBreakupData
#from breakup_data import CalculateEstimatedBreakup
from bounding_box import CalculateBoundingBox
from raster_temp import CalculateMeanTemperature
from correlation_raster import CompareIndexAndTemperature


In [3]:
# ------------------------------------------------ #
#                CLASS DEFINITION                  #
# ------------------------------------------------ #

class StudySite:
    def __init__(self,name):
        self.name = name
        self.breakup_anomaly_data = None # Dictionary of integer years (e.g. 2000) with following data subkeys: zscore_index, anomaly_days, breakup_date
        self.estimated_breakup_date = None # The MEAN breakup date for the study site across all years, in datetime format.  (Might be good to use another metric but this should be OK)
        self.enso_indices = None # Dictionary of integer years (e.g. 2000) with enso_value for each of 12 months before breakup
        self.ao_indices = None # Dictionary of integer years (e.g. 2000) with ao_value for each of 12 months before breakup
        self.bounding_box = None  # Dictionary of max_lat:value, min_lat:value, max_lon:value, min_lon:value
        self.mean_30d_raster_temps = None # Dictionary of integer years (e.g. 2000) with rasters of the mean breakup temperature across the 30 days prior to mean breakup date (we can't do actual breakup date because months have different temps)
        self.enso_correlations = None # Single raster of ENSO correlations across all years
        self.ao_correlations = None # Single raster of AO correlations across all years


    def calculate_breakup_data(self, ice_data_filepath, years_of_data):
        """ Load ice breakup data for this site from the provided icedata spreadsheet, filtering by years_of_data and the site name """

        # Dictionary of integer years (e.g. 2000) with following data subkeys: zscore_index, anomaly_days, breakup_date
        self.breakup_anomaly_data = CalculateBreakupData(years_of_data, ice_data_filepath, self.name)

        # Single value of estimated breakup date for this site (MEAN across all years)
        self.estimated_breakup_date = CalculateEstimatedBreakup(self.breakup_anomaly_data)


    def fetch_enso(self,enso_data_filepath):
        """ Pull in ENSO data as dictionary of integer years (e.g. 2000) on a MONTHLY scale for 12 months prior to estimated breakup 
            date from all years of data (excluding month with breakup date, as we are looking for time lag of 1+ months) """
        
        # Dictionary of integer years (e.g. 2000) with following data subkeys: month_name, month_number, month_value 
        self.enso_indices = ReturnENSOFetch(enso_data_filepath, self.estimated_breakup_date)


    def fetch_ao(self,ao_data_filepath):
        """ Pull in AO data as dictionary of integer years (e.g. 2000) on a MONTHLY scale for 12 months prior to estimated breakup 
            date from all years of data (excluding month with breakup date, as we are looking for time lag of 1+ months) """
        
        # Dictionary of integer years (e.g. 2000) with following data subkeys: month_name, month_number, month_value 
        self.ao_indices = ReturnAOFetch(ao_data_filepath, self.estimated_breakup_date)


    def calculate_bounding_box(self, ice_data_filepath):
        """ Calculate 100km bounding box based on study site coordinates """

        # Bounding box for this site based on lat,lon for each site in the ice data CSV
        # Dictionary of max_lat:value, min_lat:value, max_lon:value, min_lon:value
        self.bounding_box = CalculateBoundingBox(ice_data_filepath, self.name)


    def calculate_mean_raster_temps(self):
        """ Pull the raster data within the bounding box for each of the 30d prior to the given breakup date variable, then average all the rasters together to return the mean 30d temp for each year."""

        # Dictionary of integer years (e.g. 2000) paired with rasters of the mean breakup temperature across the 30 days prior to given breakup date variable for each year
        # (Note we can't do actual breakup date for each year for each site due to statistical reasons, so for each site we will find the mean breakup date and use that to find our 30d period)
        self.mean_30d_raster_temps = CalculateMeanTemperature(self.bounding_box, self.estimated_breakup_date)
       
    # For each month in our indices in the 12 months prior to breakup (excluding breakup month,) calculate the P value across all years of data
    # Single raster for each of these consisting of correlation across all years
    def calculate_ao_correlation(self):
        self.ao_correlations = CompareIndexAndTemperature(self.ao_indices, self.mean_30d_raster_temps)
    def calculate_enso_correlation(self):
        self.enso_correlations = CompareIndexAndTemperature(self.enso_indices, self.mean_30d_raster_temps)

In [4]:
# ------------------------------------------------ #
#                  PARAMETERS                      #
# ------------------------------------------------ #

# Add data here from Data folder
enso_data_filepath = os.path.join(project_root, "Data", "ENSO_index.rtf")
ao_data_filepath = os.path.join(project_root, "Data", "monthly.ao.index.txt")
ice_data_filepath = os.path.join(project_root, "Data", "ice_data.csv")

# Choose study sites here
study_site_names = ["Stebbins"]

# Generate a list of years from 2000 to 2022 - select years
years_of_data = list(range(2000, 2023))

In [8]:
# ------------------------------------------------ #
#                  MAIN CODE                      #
# ------------------------------------------------ #

# Create list of StudySite objects
sites = [StudySite(name) for name in study_site_names]

# Process data for each site
for site in sites:
    # Calculate basic site data
    #site.calculate_breakup_data(ice_data_filepath, years_of_data)
    
    site.fetch_enso(enso_data_filepath)
        
    site.fetch_ao(ao_data_filepath)
      
    site.calculate_bounding_box(ice_data_filepath)
    
    site.calculate_mean_raster_temps()
    
    site.calculate_ao_correlation()

    site.calculate_enso_correlation()

AttributeError: 'NoneType' object has no attribute 'replace'

In [ ]:
# ------------------------------------------------ #
#       CORRELATION AND PLOTTING (Placeholders)    #
# ------------------------------------------------ #

# Placeholder for correlation calculations (actual functions need to be defined)
# ao_correlation_values = CalculateAOCorrelation(ao_indices, ice_indices)
# enso_correlation_values = CalculateENSOCorrelation(enso_indices, ice_indices)

# Placeholder for charting (actual functions need to be defined)
# ChartENSOValues(enso_correlation_values)
# ChartAOValues(ao_correlation_values)

# Generate visualization charts
# ChartENSOValues(save_path=os.path.join(project_root, "Figures", f"enso_correlation_{site.name}.png"))
# ChartAOValues(save_path=os.path.join(project_root, "Figures", f"ao_correlation_{site.name}.png"))